In [1]:
import pandas as pd 
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x).rstrip('0').rstrip('.') if x != 0 else '0')
import numpy as np 
np.set_printoptions(suppress=True, precision=6)
import polars as pl 
pl.Config(set_fmt_float="full")
pl.Config(tbl_cols=1000)
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os 
import seaborn as sns
np.random.seed(22)
import torch 
from packages.utils2 import Optimize, JointOptimize
from packages.utils2 import SaveParameters, LoadParameters, JointDisplayResults, JointSaveParameters, JointLoadParameters
from packages.utils2 import prepare_data_level0, prepare_data_level1, prepare_data_level2, prepare_data_level3

device = 'cpu'

In [ ]:
# initiate your year of study :
year=2009

# load your data : 
data = pl.read_parquet(f'/your/path_to/data.parquet')

# drop intrazonal observations:
data = data.filter(pl.col('INTRAZONAL')==0)

# normalized weights:
data = data.with_columns(
    WEIGHT_hh=((pl.col('IPONDI_m')+pl.col('IPONDI_w'))/2) * (len(data)/((pl.col('IPONDI_m')+pl.col('IPONDI_w'))/2).sum()),
    WEIGHT_m=(pl.col('IPONDI_m')) * (len(data)/pl.col('IPONDI_m').sum()),
    WEIGHT_w=(pl.col('IPONDI_w')) * (len(data)/pl.col('IPONDI_w').sum())
)

# transform to pandas for the optimization step:
df = data.to_pandas()

# Overall guidelines:

- All parameters have to finish their names by _l* with * being the number of the level \
*For example : B_RES_INNERRING_l3 is the parameter of the "couronne" of the residential location in level 3*

## Steps of estimation :

1) Run the constraint model (without explanatory variables) 
2) Save the parameters of the constraint model, add some heterogeneity to the model 
3) Estimate again but with the parameters of the constraint model as initial values 
4) Save the parameters of the UNconstraint model (the one with heterogeneity)
5) Add some new variables, estimate with the UNconstraint parameters as initial values
6) Do it again until you are satified by your model 

## Tips :

- To access a variable you have to use *var('my_variable')*, with **'my_variable'** being the name of the variable in your dataset 
- To add a variable with his associated parameter, you can name the parameter as you want (**by always adding the level at the end**) \
*For example : B_DIST_BIKE2_w_l0*var('DISTANCE')**2 
- Note that we truncate the DISTANCE, TRAVEL TIME for WALK, BIKE and TC for a purpose of generalization when \
forcasting on different datasets where people could give extreme non-coherent values 

# Level 0 

### Level 0 guidelines:

- **!! IMPORTANT !!** :All parameters have to finish their names with _(gender)_l1 : **"m" for man and "w" for woman** \
*For example : B_WP_OUTERRING_m_l0 is the parameter of the "couronne" of the job location in level 1 for the man, B_WP_OUTERRING_w_l0 for woman*
- Except for *sigma_l0* which is the same for the man and the woman

### Tips:

- To add characteristics in the constant of a mode, you can add them in *XB* \
*For example : adding characteristics into XB_TC_m, is adding heterogeneity in the constant of the mode TC for the man*
- To add characteristics in the VOT (value of time) of a mode, you can add them in the *torch.exp(delta_TT_MOTO_m_l0)* term \
For example :
```python
torch.exp(delta_TT_MOTO_m_l0 + VOT_OCCP_SELFEMPLOYED_MOTO_m_l0 * var('OCCP_SELFEMPLOYED_m')
```

### Level 0 function if your year of study is after 2016 (not included)

- There is two different function for level 0 depending on the year of study : \
it is only after 2017 that INSEE has splitted MOTO and BIKE from the TWO WHEELS alternative. \
- So from 2017 there are 4 choices at the level 0 (BIKE, MOTO, TC and WALK), and before that \
there are 3 choices (TC, WALK and TWO WHEELS)

In [ ]:
def level0(params, 
           df,
           pytorch=False, 
           null_loglik=False, 
           grad=False, 
           logsum=False,
           df_length=False,
           all_sample=False):
    
    v = df['vars']

    if all_sample:
        idx_men = df['all_idx']
        idx_women = df['all_idx']
    else:
        idx_men = df['men_idx']
        idx_women = df['women_idx']

    def var(name): return v[name]

    if grad:
        params = params.clone().requires_grad_(True)
    else:
        params = torch.tensor(params, dtype=torch.float64)
        
    (sigma_l0, 
    ASC_MOTO_m_l0, ASC_BIKE_m_l0, ASC_TC_m_l0,

    B_DIST_BIKE2_m_l0, 
    B_DIST_WALK2_m_l0,
    B_TT_TC2_m_l0,
    B_TT_MOTO2_m_l0,
    delta_TT_TC_m_l0,
    delta_TT_MOTO_m_l0,
    delta_DIST_BIKE_m_l0,  

    ASC_MOTO_w_l0, ASC_BIKE_w_l0, ASC_TC_w_l0,
    
    B_DIST_BIKE2_w_l0, 
    B_DIST_WALK2_w_l0,
    B_TT_TC2_w_l0,
    B_TT_MOTO2_w_l0,
    delta_TT_TC_w_l0,
    delta_TT_MOTO_w_l0,
    delta_DIST_BIKE_w_l0
    ) = params


    # men :
    XB_TC_m = (torch.tensor(0))

    XB_MOTO_m = (torch.tensor(0))

    XB_BIKE_m = (torch.tensor(0))

    V_TC_m = (ASC_TC_m_l0 + XB_TC_m
            - torch.exp(delta_TT_TC_m_l0) * torch.min(var('TT_TC_m'), torch.tensor((180/60))) 
            + B_TT_TC2_m_l0 * (torch.min(var('TT_TC_m'), torch.tensor((180/60)))**2))
    
    V_WALK_m = (-3 * torch.min(var('DISTANCE_m'), torch.tensor(10)) 
              + B_DIST_WALK2_m_l0 * (torch.min(var('DISTANCE_m'), torch.tensor(10))**2) 
              ) 

    V_MOTO_m = (ASC_MOTO_m_l0 + XB_MOTO_m
              - torch.exp(delta_TT_MOTO_m_l0) * var('FREEFLOW_TT_m') 
              + B_TT_MOTO2_m_l0 * (var('FREEFLOW_TT_m')**2)) 
        
    V_BIKE_m = (ASC_BIKE_m_l0 + XB_BIKE_m
              - torch.exp(delta_DIST_BIKE_m_l0) * torch.min(var('DISTANCE_m'), torch.tensor(25))
              + B_DIST_BIKE2_m_l0 * (torch.min(var('DISTANCE_m'), torch.tensor(25))**2)
              )
    
    # women:
    XB_TC_w = (torch.tensor(0))

    XB_MOTO_w = (torch.tensor(0))

    XB_BIKE_w = (torch.tensor(0))

    # example of how heterogeneity can be added to the constant of BIKE for woman:
    # XB_BIKE_w = (B_AGE40_BIKE_w_l0 * ((var('AGED_w')-40)/10) 
    #            + B_AGE40_BIKE2_w_l0 * (((var('AGED_w')-40)/10)**2))

    # ------- Utility functions

    V_TC_w = (ASC_TC_w_l0 + XB_TC_w
            - torch.exp(delta_TT_TC_w_l0) * torch.min(var('TT_TC_w'), torch.tensor((180/60))) 
            + B_TT_TC2_w_l0 * (torch.min(var('TT_TC_w')**2, torch.tensor((180/60)))**2))

    V_MOTO_w = (ASC_MOTO_w_l0 + XB_MOTO_w
              - torch.exp(delta_TT_MOTO_w_l0) * var('FREEFLOW_TT_w') 
              + B_TT_MOTO2_w_l0 * (var('FREEFLOW_TT_w')**2)) 
    
    V_WALK_w = (-3 * torch.min(var('DISTANCE_w'), torch.tensor(10)) 
              + B_DIST_WALK2_w_l0 * (torch.min(var('DISTANCE_w')**2, torch.tensor(10))**2) 
              ) 
    
    V_BIKE_w = (ASC_BIKE_w_l0 + XB_BIKE_w
              - torch.exp(delta_DIST_BIKE_w_l0) * torch.min(var('DISTANCE_w'), torch.tensor(25))
              + B_DIST_BIKE2_w_l0 * (torch.min(var('DISTANCE_w')**2, torch.tensor(25))**2)
              )
    
    # --------------- Men's probabilities
    # https://stats.stackexchange.com/questions/304758/softmax-overflow
    V_men_stack = torch.stack([V_TC_m[idx_men] / sigma_l0, 
                            V_MOTO_m[idx_men] / sigma_l0, 
                            V_WALK_m[idx_men] / sigma_l0, 
                            V_BIKE_m[idx_men] / sigma_l0], dim=1)
    max_V_men = V_men_stack.max()
    exp_V_men = torch.exp(V_men_stack - max_V_men)
    sum_exp_men = exp_V_men.sum(dim=1)

    P_TC_m, P_MOTO_m, P_WALK_m, P_BIKE_m = exp_V_men.T / sum_exp_men

    # --------------- Women's probabilities
    # https://stats.stackexchange.com/questions/304758/softmax-overflow
    V_women_stack = torch.stack([V_TC_w[idx_women] / sigma_l0, 
                            V_MOTO_w[idx_women] / sigma_l0, 
                            V_WALK_w[idx_women] / sigma_l0, 
                            V_BIKE_w[idx_women] / sigma_l0], dim=1)
    max_V_women = V_women_stack.max()
    exp_V_women = torch.exp(V_women_stack - max_V_women)
    sum_exp_women = exp_V_women.sum(dim=1)

    P_TC_w, P_MOTO_w, P_WALK_w, P_BIKE_w = exp_V_women.T / sum_exp_women

    # --------------- Choices

    choices = df['df']
    
    Choice_TC_m = torch.tensor((choices['COMMUTE_MODE_m']=='PUBLIC').astype(int).values, dtype=torch.float64)
    Choice_WALK_m = torch.tensor((choices['COMMUTE_MODE_m']=='WALK').astype(int).values, dtype=torch.float64)
    Choice_MOTO_m = torch.tensor((choices['COMMUTE_MODE_m']=='MOTO').astype(int).values, dtype=torch.float64)
    Choice_BIKE_m = torch.tensor((choices['COMMUTE_MODE_m']=='BIKE').astype(int).values, dtype=torch.float64)

    Choice_TC_w = torch.tensor((choices['COMMUTE_MODE_w']=='PUBLIC').astype(int).values, dtype=torch.float64)
    Choice_WALK_w = torch.tensor((choices['COMMUTE_MODE_w']=='WALK').astype(int).values, dtype=torch.float64)
    Choice_MOTO_w = torch.tensor((choices['COMMUTE_MODE_w']=='MOTO').astype(int).values, dtype=torch.float64)
    Choice_BIKE_w = torch.tensor((choices['COMMUTE_MODE_w']=='BIKE').astype(int).values, dtype=torch.float64)

    weights_m = var('WEIGHT_m')
    weights_m = weights_m[idx_men]
    weights_w = var('WEIGHT_w')
    weights_w = weights_w[idx_women]
    
    if null_loglik:
        null_LL = torch.sum(weights_m * Choice_TC_m[idx_men] * torch.log(torch.sum(Choice_TC_m[idx_men])/len(idx_men))) + \
                torch.sum(weights_m * Choice_MOTO_m[idx_men] * torch.log(torch.sum(Choice_MOTO_m[idx_men])/len(idx_men))) + \
                torch.sum(weights_m * Choice_WALK_m[idx_men] * torch.log(torch.sum(Choice_WALK_m[idx_men])/len(idx_men))) + \
                torch.sum(weights_m * Choice_BIKE_m[idx_men] * torch.log(torch.sum(Choice_BIKE_m[idx_men])/len(idx_men))) + \
                torch.sum(weights_w * Choice_TC_w[idx_women] * torch.log(torch.sum(Choice_TC_w[idx_women])/len(idx_women))) + \
                torch.sum(weights_w * Choice_MOTO_w[idx_women] * torch.log(torch.sum(Choice_MOTO_w[idx_women])/len(idx_women))) + \
                torch.sum(weights_w * Choice_WALK_w[idx_women] * torch.log(torch.sum(Choice_WALK_w[idx_women])/len(idx_women))) + \
                torch.sum(weights_w * Choice_BIKE_w[idx_women] * torch.log(torch.sum(Choice_BIKE_w[idx_women])/len(idx_women)))

        return -null_LL

    epsilon = 1e-30  


    LL = torch.sum(weights_m * Choice_TC_m[idx_men] * torch.log(torch.clamp(P_TC_m, min=epsilon))) + \
        torch.sum(weights_m * Choice_MOTO_m[idx_men] * torch.log(torch.clamp(P_MOTO_m, min=epsilon))) + \
        torch.sum(weights_m * Choice_WALK_m[idx_men] * torch.log(torch.clamp(P_WALK_m, min=epsilon))) + \
        torch.sum(weights_m * Choice_BIKE_m[idx_men] * torch.log(torch.clamp(P_BIKE_m, min=epsilon))) + \
        torch.sum(weights_w * Choice_TC_w[idx_women] * torch.log(torch.clamp(P_TC_w, min=epsilon))) + \
        torch.sum(weights_w * Choice_MOTO_w[idx_women] * torch.log(torch.clamp(P_MOTO_w, min=epsilon))) + \
        torch.sum(weights_w * Choice_WALK_w[idx_women] * torch.log(torch.clamp(P_WALK_w, min=epsilon))) + \
        torch.sum(weights_w * Choice_BIKE_w[idx_women] * torch.log(torch.clamp(P_BIKE_w, min=epsilon)))

    if pytorch:
        return -LL
        
    if logsum:
        LS_m = sigma_l0 * torch.log(
            torch.exp(V_TC_m / sigma_l0) + torch.exp(V_MOTO_m / sigma_l0) 
            + torch.exp(V_WALK_m / sigma_l0) + torch.exp(V_BIKE_m / sigma_l0) 
        )
        LS_w = sigma_l0 * torch.log(
            torch.exp(V_TC_w / sigma_l0) + torch.exp(V_MOTO_w / sigma_l0) 
            + torch.exp(V_WALK_w / sigma_l0) + torch.exp(V_BIKE_w / sigma_l0) 
        )
        return LS_m, LS_w
    
    if df_length:
        return len(idx_men) + len(idx_women)
        
    return (-LL).detach().numpy()

### Level 0 function if your year of study is before 2016 (included)

In [ ]:
def level0(params, 
           df,
           pytorch=False, 
           null_loglik=False, 
           grad=False, 
           logsum=False,
           df_length=False,
           all_sample=False):
    
    v = df['vars']

    if all_sample:
        idx_men = df['all_idx']
        idx_women = df['all_idx']
    else:
        idx_men = df['men_idx']
        idx_women = df['women_idx']

    def var(name): return v[name]

    if grad:
        params = params.clone().requires_grad_(True)
    else:
        params = torch.tensor(params, dtype=torch.float64)
        
    (
        sigma_l0, 
        ASC_2R_m_l0, ASC_TC_m_l0,

        B_DIST_2R2_m_l0, 
        B_DIST_WALK2_m_l0,
        B_TT_TC2_m_l0,
        delta_TT_TC_m_l0,
        delta_DIST_2R_m_l0,  

        ASC_2R_w_l0, ASC_TC_w_l0,

        B_DIST_2R2_w_l0, 
        B_DIST_WALK2_w_l0,
        B_TT_TC2_w_l0,
        delta_TT_TC_w_l0,
        delta_DIST_2R_w_l0,

    ) = params


    # men :
    XB_TC_m = (torch.tensor(0))

    XB_2R_m = (torch.tensor(0))

    V_TC_m = (ASC_TC_m_l0 + XB_TC_m
            - torch.exp(delta_TT_TC_m_l0) * torch.min(var('TT_TC_m'), torch.tensor((180/60))) 
            + B_TT_TC2_m_l0 * (torch.min(var('TT_TC_m'), torch.tensor((180/60)))**2))
    
    V_WALK_m = (-3 * torch.min(var('DISTANCE_m'), torch.tensor(10)) 
              + B_DIST_WALK2_m_l0 * (torch.min(var('DISTANCE_m'), torch.tensor(10))**2) 
              ) 
        
    V_2R_m = (ASC_2R_m_l0 + XB_2R_m
              - torch.exp(delta_DIST_2R_m_l0) * torch.min(var('DISTANCE_m'), torch.tensor(25))
              + B_DIST_2R2_m_l0 * (torch.min(var('DISTANCE_m'), torch.tensor(25))**2)
              )
    
    # women:
    XB_TC_w = (torch.tensor(0))

    XB_2R_w = (torch.tensor(0))

    # example of how heterogeneity can be added to the constant of TWO WHEELS (2R=deux roues) for woman:
    # XB_2R_w = (B_AGE40_2R_w_l0 * ((var('AGED_w')-40)/10) 
    #            + B_AGE40_2R2_w_l0 * (((var('AGED_w')-40)/10)**2))

    # ------- Utility functions

    V_TC_w = (ASC_TC_w_l0 + XB_TC_w
            - torch.exp(delta_TT_TC_w_l0) * torch.min(var('TT_TC_w'), torch.tensor((180/60))) 
            + B_TT_TC2_w_l0 * (torch.min(var('TT_TC_w'), torch.tensor((180/60)))**2))
    
    V_WALK_w = (-3 * torch.min(var('DISTANCE_w'), torch.tensor(10)) 
              + B_DIST_WALK2_w_l0 * (torch.min(var('DISTANCE_w'), torch.tensor(10))**2) 
              ) 
    
    V_2R_w = (ASC_2R_w_l0 + XB_2R_w
              - torch.exp(delta_DIST_2R_w_l0) * torch.min(var('DISTANCE_w'), torch.tensor(25))
              + B_DIST_2R2_w_l0 * (torch.min(var('DISTANCE_w'), torch.tensor(25))**2)
              )
    
    # --------------- Men's probabilities
    # https://stats.stackexchange.com/questions/304758/softmax-overflow
    V_men_stack = torch.stack([V_TC_m[idx_men] / sigma_l0, 
                            V_WALK_m[idx_men] / sigma_l0, 
                            V_2R_m[idx_men] / sigma_l0], dim=1)
    max_V_men = V_men_stack.max()
    exp_V_men = torch.exp(V_men_stack - max_V_men)
    sum_exp_men = exp_V_men.sum(dim=1)

    P_TC_m, P_WALK_m, P_2R_m = exp_V_men.T / sum_exp_men

    # --------------- Women's probabilities
    # https://stats.stackexchange.com/questions/304758/softmax-overflow
    V_women_stack = torch.stack([V_TC_w[idx_women] / sigma_l0, 
                            V_WALK_w[idx_women] / sigma_l0, 
                            V_2R_w[idx_women] / sigma_l0], dim=1)
    max_V_women = V_women_stack.max()
    exp_V_women = torch.exp(V_women_stack - max_V_women)
    sum_exp_women = exp_V_women.sum(dim=1)

    P_TC_w, P_WALK_w, P_2R_w = exp_V_women.T / sum_exp_women

    # --------------- Choices

    choices = df['df']
    
    Choice_TC_m = torch.tensor((choices['COMMUTE_MODE_m']=='PUBLIC').astype(int).values, dtype=torch.float64)
    Choice_WALK_m = torch.tensor((choices['COMMUTE_MODE_m']=='WALK').astype(int).values, dtype=torch.float64)
    Choice_2R_m = torch.tensor((choices['COMMUTE_MODE_m']=='TWO_WHEELS').astype(int).values, dtype=torch.float64)

    Choice_TC_w = torch.tensor((choices['COMMUTE_MODE_w']=='PUBLIC').astype(int).values, dtype=torch.float64)
    Choice_WALK_w = torch.tensor((choices['COMMUTE_MODE_w']=='WALK').astype(int).values, dtype=torch.float64)
    Choice_2R_w = torch.tensor((choices['COMMUTE_MODE_w']=='TWO_WHEELS').astype(int).values, dtype=torch.float64)

    weights_m = var('WEIGHT_m')
    weights_m = weights_m[idx_men]
    weights_w = var('WEIGHT_w')
    weights_w = weights_w[idx_women]
    
    if null_loglik:

        null_LL = torch.sum(weights_m * Choice_TC_m[idx_men] * torch.log(torch.sum(Choice_TC_m[idx_men])/len(idx_men))) + \
            torch.sum(weights_m * Choice_2R_m[idx_men] * torch.log(torch.sum(Choice_2R_m[idx_men])/len(idx_men))) + \
            torch.sum(weights_m * Choice_WALK_m[idx_men] * torch.log(torch.sum(Choice_WALK_m[idx_men])/len(idx_men))) + \
            torch.sum(weights_w * Choice_TC_w[idx_women] * torch.log(torch.sum(Choice_TC_w[idx_women])/len(idx_women))) + \
            torch.sum(weights_w * Choice_2R_w[idx_women] * torch.log(torch.sum(Choice_2R_w[idx_women])/len(idx_women))) + \
            torch.sum(weights_w * Choice_WALK_w[idx_women] * torch.log(torch.sum(Choice_WALK_w[idx_women])/len(idx_women))) 
        return -null_LL

    epsilon = 1e-30  


    LL = torch.sum(weights_m * Choice_TC_m[idx_men] * torch.log(torch.clamp(P_TC_m, min=epsilon))) + \
        torch.sum(weights_m * Choice_2R_m[idx_men] * torch.log(torch.clamp(P_2R_m, min=epsilon))) + \
        torch.sum(weights_m * Choice_WALK_m[idx_men] * torch.log(torch.clamp(P_WALK_m, min=epsilon))) + \
        torch.sum(weights_w * Choice_TC_w[idx_women] * torch.log(torch.clamp(P_TC_w, min=epsilon))) + \
        torch.sum(weights_w * Choice_2R_w[idx_women] * torch.log(torch.clamp(P_2R_w, min=epsilon))) + \
        torch.sum(weights_w * Choice_WALK_w[idx_women] * torch.log(torch.clamp(P_WALK_w, min=epsilon))) 

    if pytorch:
        return -LL
        
    if logsum:
        LS_m = sigma_l0 * torch.log(
            torch.exp(V_TC_m / sigma_l0) 
            + torch.exp(V_WALK_m / sigma_l0) + torch.exp(V_2R_m / sigma_l0) 
        )
        LS_w = sigma_l0 * torch.log(
            torch.exp(V_TC_w / sigma_l0) 
            + torch.exp(V_WALK_w / sigma_l0) + torch.exp(V_2R_w / sigma_l0) 
        )
        return LS_m, LS_w
    
    if df_length:
        return len(idx_men) + len(idx_women)
        
    return (-LL).detach().numpy()

### Tips:
- The function will stop optimizing after 5000 iterations, you modify it as you wish 
- You can modify the **gtol** parameter to get more precision on the estimation but \
it will be costly in terms of computation time
- If your estimation takes too much time I suggest you to increase the **gtol** by 0.5 

### How to estimate :

- Run the first estimation by leaving the **initial_values** commented 
- Then you can see the estimated parameters in the **summary_level0** table 
- Save the parameters with the **SaveParameters** function => parameters will be saved in the parameters folder that is on the same folder as your code
- Then you can add some new variables to your objective function, and start a new estimation with the last parameters as **initial_values**

In [ ]:
summary_level0, parameters_level0 = Optimize(level0, 
         prepare_data_level0(df, year=year), 
        #  initial_values='Level0_CONSTRAINT',
         max_iter=5000,
         gtol=1,
         display_results=True)

Optimizing:   0%|          | 0/5000 [00:00<?, ?it/s]

Optimizing:   9%|▊         | 431/5000 [01:26<15:02,  5.07it/s, Objective Value=71188.10487]

Convergence: True


Optimizing:   9%|▊         | 431/5000 [01:29<15:47,  4.82it/s, Objective Value=71188.10487]


In [ ]:
SaveParameters(level0,
               parameters_level0,
               excel=True, # if True : this will give you the summary_level0 table in an excel file in the parameters folder 
               data=prepare_data_level0(df, year=year),
               file_name='Level0_CONSTRAINT')

# Level 1 

## Parameter names convention :

- for the parameters of the Pareto weight (Lambda) : begin with **L_**
- for the parameters of the different constants : begin with **B_**
- for the parameters of the value of time : begin with **votCA_** if the value of time of car alone, **votCP_** if car passenger, etc.
- the end of each parameter should always follow the rules of each level : so, here each parameter ends with : **_l1**

### Tips :
- Here we start by computing the *logsum* resulting from level 0. Be careful to load the right parameters to compute it \
otherwise you will end up with a wrong *logsum* term 
- This level is the **hardest to identify** : I recommand you to add variables in the Lambda, \
the constants of OTHERS (being the case of choosing other modes than car), and the VOT of car alone at the same time 
- *ASC_Cp_m_l1*, *ASC_Cp_w_l1*, *ASC_Cd_m_l1*, *ASC_Cd_w_l1* and \
*delta_Cp_m_l1*, *delta_Cp_w_l1*, *delta_Cd_m_l1*, *delta_Cd_w_l1* are the **hardest to identify**

In [ ]:
# load the parameters of the final model in level 0 (so the last UNconstraint model that you've estimated)
loaded_parameters = LoadParameters(
                func=level0, 
                file_name='Level0_UNCONSTRAINT') 

# this adds the logsums of the man and the woman to the dataset
df['LS_m'], df['LS_w'] = level0(loaded_parameters, prepare_data_level0(df, year=year), logsum=True)


def level1(params, 
                  df, 
                  pytorch=False, 
                  grad=False, 
                  null_loglik=False, 
                  df_length=False,
                  pareto_weight=False,
                  logsum=False,
                  utility_nestB=False,
                  all_sample=False,
                  year=year):
        v = df["vars"]

        if all_sample:
                idx1 = df["all_idx"]
                idx2 = df["all_idx"]
        else:
                idx1 = df["is_one_car_idx"]
                idx2 = df["is_multi_car_idx"]

        def var(name): return v[name]

        if grad:
                params = params.clone().requires_grad_(True)
        else:
                params = torch.tensor(params, dtype=torch.float64)
    
        (
        sigma_1c_l1, sigma_2c_l1,

        ASC_Cp_m_l1, ASC_Cp_w_l1,
        ASC_Cd_m_l1, ASC_Cd_w_l1,

        ASC_B_w_l1, delta_Ca_w_l1, delta_Cd_w_l1, delta_Cp_w_l1,
        ASC_B_m_l1, delta_Ca_m_l1, delta_Cp_m_l1, delta_Cd_m_l1,

        ) = params

        Xb_Lambda = (torch.tensor(0))

        Lambda = 1 / (1 + torch.exp(-Xb_Lambda))

        Xb_m_OTHERS = (torch.tensor(0))
                       
        Xb_w_OTHERS = (torch.tensor(0))

        Xb_m_CP = (torch.tensor(0))

        Xb_w_CP = (torch.tensor(0))

        Xb_m_CD = (torch.tensor(0))

        Xb_w_CD = (torch.tensor(0))

        # individual vot and utilities:
        VOT_Ca_m = torch.exp(delta_Ca_m_l1)
        
        VOT_Ca_w = torch.exp(delta_Ca_w_l1)

        V_Ca_m = -VOT_Ca_m*var('TT_VP_m')
        V_Ca_w = -VOT_Ca_w*var('TT_VP_w')

        V_Cp_m = (ASC_Cp_m_l1 + Xb_m_CP) - VOT_Ca_m*torch.exp(delta_Cp_m_l1)*var('TT_VP_m') 
        V_Cp_w = (ASC_Cp_w_l1 + Xb_w_CP) - VOT_Ca_w*torch.exp(delta_Cp_w_l1)*var('TT_VP_w') 

        V_Cd_m = (ASC_Cd_m_l1 + Xb_m_CD) - VOT_Ca_m*torch.exp(delta_Cd_m_l1)*var('TT_VP_w') - VOT_Ca_m*var('WOMANtowardsMAN')
        V_Cd_w = (ASC_Cd_w_l1 + Xb_w_CD) - VOT_Ca_w*torch.exp(delta_Cd_w_l1)*var('TT_VP_m') - VOT_Ca_w*var('MANtowardsWOMAN')

        V_B_m = (ASC_B_m_l1 + Xb_m_OTHERS) + var('LS_m')
        V_B_w = (ASC_B_w_l1 + Xb_w_OTHERS) + var('LS_w')

        # Utilitaires collectifs : (extrait)
        V_CaCa = Lambda * V_Ca_w + (1 - Lambda) * V_Ca_m
        V_BB = Lambda * V_B_w + (1 - Lambda) * V_B_m
        V_CaB = Lambda * V_Ca_w + (1 - Lambda) * V_B_m
        V_BCa = Lambda * V_B_w + (1 - Lambda) * V_Ca_m
        V_CdCp = Lambda * V_Cd_w + (1 - Lambda) * V_Cp_m
        V_CpCd = Lambda * V_Cp_w + (1 - Lambda) * V_Cd_m

        # -------- One-car probabilities
        V_1c_stack = torch.stack([
                V_BB[idx1] / sigma_1c_l1,
                V_CaB[idx1] / sigma_1c_l1,
                V_BCa[idx1] / sigma_1c_l1,
                V_CdCp[idx1] / sigma_1c_l1,
                V_CpCd[idx1] / sigma_1c_l1
        ], dim=1)
        max_V_1c = V_1c_stack.max()
        exp_V_1c = torch.exp(V_1c_stack - max_V_1c)
        sum_exp_1c = exp_V_1c.sum(dim=1)

        P_BB_1c, P_CaB_1c, P_BCa_1c, P_CdCp_1c, P_CpCd_1c = exp_V_1c.T / sum_exp_1c

        # -------- Two-car probabilities
        V_2c_stack = torch.stack([
                V_CaCa[idx2] / sigma_2c_l1,
                V_BB[idx2] / sigma_2c_l1,
                V_CaB[idx2] / sigma_2c_l1,
                V_BCa[idx2] / sigma_2c_l1,
                V_CdCp[idx2] / sigma_2c_l1,
                V_CpCd[idx2] / sigma_2c_l1
        ], dim=1)
        max_V_2c = V_2c_stack.max()
        exp_V_2c = torch.exp(V_2c_stack - max_V_2c)
        sum_exp_2c = exp_V_2c.sum(dim=1)

        P_CaCa_2c, P_BB_2c, P_CaB_2c, P_BCa_2c, P_CdCp_2c, P_CpCd_2c = exp_V_2c.T / sum_exp_2c

        # -------- Choices
        choices = df['df']

        if year>2016:
                Choice_CC = ((choices['TRANS_w'] == 5) & (choices['TRANS_m'] == 5)).to_numpy().astype(float)
                Choice_BB = ((choices['TRANS_w'].isin([2, 3, 4, 6])) & (choices['TRANS_m'].isin([2, 3, 4, 6]))).to_numpy().astype(float)
                Choice_CaB = ((choices['TRANS_w'] == 5) & (choices['TRANS_m'].isin([2, 3, 4, 6]))).to_numpy().astype(float)
                Choice_BCa = ((choices['TRANS_w'].isin([2, 3, 4, 6])) & (choices['TRANS_m'] == 5)).to_numpy().astype(float)
        else:
                Choice_CC = ((choices['TRANS_w'] == 4) & (choices['TRANS_m'] == 4)).to_numpy().astype(float)
                Choice_BB = ((choices['TRANS_w'].isin([2, 3, 5])) & (choices['TRANS_m'].isin([2, 3, 5]))).to_numpy().astype(float)
                Choice_CaB = ((choices['TRANS_w'] == 4) & (choices['TRANS_m'].isin([2, 3, 5]))).to_numpy().astype(float)
                Choice_BCa = ((choices['TRANS_w'].isin([2, 3, 5])) & (choices['TRANS_m'] == 4)).to_numpy().astype(float)


        w_1c = var('WEIGHT_hh')
        w_1c = w_1c[idx1]
        w_2c = var('WEIGHT_hh')
        w_2c = w_2c[idx2]

        epsilon = 1e-30

        LL_1c = (
        torch.sum(w_1c * torch.tensor(Choice_CC)[idx1] * torch.log(torch.clamp(P_CdCp_1c + P_CpCd_1c, min=epsilon))) +
        torch.sum(w_1c * torch.tensor(Choice_BB)[idx1] * torch.log(torch.clamp(P_BB_1c, min=epsilon))) +
        torch.sum(w_1c * torch.tensor(Choice_CaB)[idx1] * torch.log(torch.clamp(P_CaB_1c, min=epsilon))) +
        torch.sum(w_1c * torch.tensor(Choice_BCa)[idx1] * torch.log(torch.clamp(P_BCa_1c, min=epsilon)))
        )

        LL_2c = (
        torch.sum(w_2c * torch.tensor(Choice_CC)[idx2] * torch.log(torch.clamp(P_CaCa_2c + P_CdCp_2c + P_CpCd_2c, min=epsilon))) +
        torch.sum(w_2c * torch.tensor(Choice_BB)[idx2] * torch.log(torch.clamp(P_BB_2c, min=epsilon))) +
        torch.sum(w_2c * torch.tensor(Choice_CaB)[idx2] * torch.log(torch.clamp(P_CaB_2c, min=epsilon))) +
        torch.sum(w_2c * torch.tensor(Choice_BCa)[idx2] * torch.log(torch.clamp(P_BCa_2c, min=epsilon)))
        )

        if pytorch:
                return -(LL_1c + LL_2c)
        if df_length:
                return len(V_B_w)
        
        if not pytorch and null_loglik:
                n_1c = len(idx1)
                n_2c = len(idx2)

                if year>2016:
                        Choice_CC = torch.tensor(((choices['TRANS_w'] == 5) & (choices['TRANS_m'] == 5)).astype(float).values)
                        Choice_BB = torch.tensor(((choices['TRANS_w'].isin([2, 3, 4, 6])) & (choices['TRANS_m'].isin([2, 3, 4, 6]))).astype(float).values)
                        Choice_CaB = torch.tensor(((choices['TRANS_w'] == 5) & (choices['TRANS_m'].isin([2, 3, 4, 6]))).astype(float).values)
                        Choice_BCa = torch.tensor(((choices['TRANS_w'].isin([2, 3, 4, 6])) & (choices['TRANS_m'] == 5)).astype(float).values)
                else:
                        Choice_CC = torch.tensor(((choices['TRANS_w'] == 4) & (choices['TRANS_m'] == 4 )).astype(float).values)
                        Choice_BB = torch.tensor(((choices['TRANS_w'].isin([2, 3, 5])) & (choices['TRANS_m'].isin([2, 3, 5]))).astype(float).values)
                        Choice_CaB = torch.tensor(((choices['TRANS_w'] == 4) & (choices['TRANS_m'].isin([2, 3, 5]))).astype(float).values)
                        Choice_BCa = torch.tensor(((choices['TRANS_w'].isin([2, 3, 5])) & (choices['TRANS_m'] == 4)).astype(float).values)

                Choice_CC_1c, Choice_BB_1c, Choice_CaB_1c, Choice_BCa_1c = (
                Choice_CC[idx1], Choice_BB[idx1], Choice_CaB[idx1], Choice_BCa[idx1])
                Choice_CC_2c, Choice_BB_2c, Choice_CaB_2c, Choice_BCa_2c = (
                Choice_CC[idx2], Choice_BB[idx2], Choice_CaB[idx2], Choice_BCa[idx2])

                # observed shares
                share_CC_1c = torch.sum(Choice_CC_1c) / n_1c
                share_BB_1c = torch.sum(Choice_BB_1c) / n_1c
                share_CaB_1c = torch.sum(Choice_CaB_1c) / n_1c
                share_BCa_1c = torch.sum(Choice_BCa_1c) / n_1c

                share_CC_2c = torch.sum(Choice_CC_2c) / n_2c
                share_BB_2c = torch.sum(Choice_BB_2c) / n_2c
                share_CaB_2c = torch.sum(Choice_CaB_2c) / n_2c
                share_BCa_2c = torch.sum(Choice_BCa_2c) / n_2c

                w_1c = var('WEIGHT_hh')
                w_1c = w_1c[idx1]
                w_2c = var('WEIGHT_hh')
                w_2c = w_2c[idx2]

                null_LL_1c = (
                torch.sum(w_1c * Choice_CC_1c * torch.log(torch.clamp(share_CC_1c, min=epsilon))) +
                torch.sum(w_1c * Choice_BB_1c * torch.log(torch.clamp(share_BB_1c, min=epsilon))) +
                torch.sum(w_1c * Choice_CaB_1c * torch.log(torch.clamp(share_CaB_1c, min=epsilon))) +
                torch.sum(w_1c * Choice_BCa_1c * torch.log(torch.clamp(share_BCa_1c, min=epsilon)))
                )

                null_LL_2c = (
                torch.sum(w_2c * Choice_CC_2c * torch.log(torch.clamp(share_CC_2c, min=epsilon))) +
                torch.sum(w_2c * Choice_BB_2c * torch.log(torch.clamp(share_BB_2c, min=epsilon))) +
                torch.sum(w_2c * Choice_CaB_2c * torch.log(torch.clamp(share_CaB_2c, min=epsilon))) +
                torch.sum(w_2c * Choice_BCa_2c * torch.log(torch.clamp(share_BCa_2c, min=epsilon)))
                )

                return -(null_LL_1c + null_LL_2c)
        
        if pareto_weight:
                return Lambda
        
        if logsum:
                LS_1c = sigma_1c_l1 * torch.log(
                        torch.exp(V_BB/sigma_1c_l1) + torch.exp(V_CaB/sigma_1c_l1) 
                        + torch.exp(V_BCa/sigma_1c_l1) + torch.exp(V_CpCd/sigma_1c_l1) + torch.exp(V_CdCp/sigma_1c_l1)
                )
                LS_2c = sigma_2c_l1 * torch.log(
                        torch.exp(V_CaCa/sigma_2c_l1) + torch.exp(V_BB/sigma_2c_l1) + torch.exp(V_CaB/sigma_2c_l1) 
                        + torch.exp(V_BCa/sigma_2c_l1) + torch.exp(V_CpCd/sigma_2c_l1) + torch.exp(V_CdCp/sigma_2c_l1)
                )
                return (LS_1c).detach().numpy(), (LS_2c).detach().numpy()
        if utility_nestB:
                return V_BB.detach().numpy()
        
        return -(LL_1c + LL_2c).detach().numpy()

In [ ]:
summary_level1, parameters_level1 = Optimize(level1, 
         prepare_data_level1(df), 
        #  initial_values='Level1_CONSTRAINT',
         max_iter=5000,
         gtol=1,
         display_results=True)

In [ ]:
SaveParameters(level1,
               parameters_level1,
               excel=True,
               data=prepare_data_level1(df),
               file_name='Level1_CONSTRAINT')

# Level 2

In [ ]:
loaded_parameters = LoadParameters(
                func=level1, 
                file_name='Level1_UNCONSTRAINT')

df['LS_1c'], df['LS_2c'] = level1(loaded_parameters, prepare_data_level1(df), logsum=True)

def level2(params, 
                  df, 
                  pytorch=False, 
                  grad=False, 
                  null_loglik=False, 
                  df_length=False,
                  logsum=False,
                  all_sample=False):
        v = df["vars"]

        if all_sample:
                idx = df["all_idx"]
        else:
                idx = df["at_least_one_car_idx"]

        def var(name): return v[name]

        if grad:
                params = params.clone().requires_grad_(True)
        else:
                params = torch.tensor(params, dtype=torch.float64)

        (sigma_l2,
        
        ASC_2c_l2) = params

        XB_2c = (torch.tensor(0))
        
        V_1car = var('LS_1c') 
        V_2car = (ASC_2c_l2 + XB_2c + var('LS_2c')) 

        # -------- Probabilities
        V_stack = torch.stack([
                V_1car[idx] / sigma_l2,
                V_2car[idx] / sigma_l2
        ], dim=1)
        max_V = V_stack.max()
        exp_V = torch.exp(V_stack - max_V)
        sum_exp = exp_V.sum(dim=1)

        P_1car, P_2car = exp_V.T / sum_exp

        choice = df['df']
        CHOICE_1CAR = torch.tensor((choice['VOIT'] == 1).astype(int), dtype=torch.float64)
        CHOICE_2CAR = torch.tensor((choice['VOIT'] >= 2).astype(int), dtype=torch.float64)

        w = var('WEIGHT_hh')
        w = w[idx]

        epsilon=1e-30
        LL = (torch.sum(w * CHOICE_1CAR[idx] * torch.log(torch.clamp(P_1car, min=epsilon))) +
          torch.sum(w * CHOICE_2CAR[idx] * torch.log(torch.clamp(P_2car, min=epsilon))))
        
        if pytorch:
                return -LL
        
        if null_loglik:
                null_LL = (torch.sum(w * CHOICE_1CAR[idx] * torch.log(torch.sum(CHOICE_1CAR[idx])/len(idx)))
                        + torch.sum(w * CHOICE_2CAR[idx] * torch.log(torch.sum(CHOICE_2CAR[idx])/len(idx))))
                return -null_LL
        
        if df_length:
                return len(idx)
        
        if logsum:
                LS_level2 = sigma_l2 * torch.log(
                        torch.exp(V_1car/sigma_l2) + torch.exp(V_2car/sigma_l2)
                )
                return (LS_level2).detach().numpy()
        
        return (-LL).detach().numpy()

In [ ]:
summary_level2, parameters_level2 = Optimize(level2, 
        prepare_data_level2(df), 
        #  initial_values='Level2_CONSTRAINT',
        max_iter=5000,
        gtol=1,
        display_results=True)

In [ ]:
SaveParameters(level2,
               parameters_level2,
               excel=True,
               data=prepare_data_level2(df),
               file_name='Level2_CONSTRAINT')

# Level 3

In [ ]:
loaded_parameters = LoadParameters(
                func=level2, 
                file_name='Level2_UNCONSTRAINT')

df['LS_car'] = level2(loaded_parameters, prepare_data_level2(df), logsum=True)

loaded_parameters = LoadParameters(
                func=level1, 
                file_name='Level1_UNCONSTRAINT')

df['V_BB'] = level1(loaded_parameters, prepare_data_level1(df), utility_nestB=True)

def level3(params, 
                df, 
                pytorch=False, 
                grad=False, 
                null_loglik=False, 
                df_length=False,
                logsum=False):
        v = df["vars"]

        def var(name): return v[name]

        if grad:
                params = params.clone().requires_grad_(True)
        else:
                params = torch.tensor(params, dtype=torch.float64)

        (sigma_l3,

        ASC_car_l3) = params

        XB_car = (torch.tensor(0))

        V_NoCar = var('V_BB') 
        V_Car = ASC_car_l3 + XB_car + var('LS_car')

        V_stack = torch.stack([
                V_NoCar/sigma_l3, 
                V_Car/sigma_l3], dim=1)
        max_V = V_stack.max()
        exp_V = torch.exp(V_stack - max_V)
        sum_exp = exp_V.sum(dim=1)

        P_NoCar, P_Car = exp_V.T / sum_exp

        choice = df['df']
        CHOICE_NOCAR = torch.tensor((choice['VOIT'] == 0).astype(int), dtype=torch.float64)
        CHOICE_CAR = torch.tensor((choice['VOIT'] >= 1).astype(int), dtype=torch.float64)

        epsilon=1e-30
        w = var('WEIGHT_hh')

        LL = (torch.sum(w * CHOICE_NOCAR * torch.log(torch.clamp(P_NoCar, min=epsilon))) +
                torch.sum(w * CHOICE_CAR * torch.log(torch.clamp(P_Car, min=epsilon))))

        if pytorch:
                return -LL

        if null_loglik:
                null_LL = (torch.sum(w * CHOICE_NOCAR * torch.log(torch.sum(CHOICE_NOCAR)/len(CHOICE_NOCAR)))
                        + torch.sum(w * CHOICE_CAR * torch.log(torch.sum(CHOICE_CAR)/len(CHOICE_CAR))))
                return -null_LL

        if df_length:
                return len(df['df'])

        if logsum:
                LS_level3 = sigma_l3 * torch.log(
                torch.exp(V_NoCar/sigma_l3) + torch.exp(V_Car/sigma_l3)
                )
                return (LS_level3).detach().numpy()

        return (-LL).detach().numpy()

In [ ]:
summary_level3, parameters_level3 = Optimize(level3, 
         prepare_data_level3(df), 
        #  initial_values='Level3_CONSTRAINT',
         display_results=True)

In [ ]:
SaveParameters(level3,
               parameters_level3,
               excel=True,
               data=prepare_data_level3(df),
               file_name='Level3_CONSTRAINT')

# Simultaneous estimation 

In [ ]:
def SimultaneousNestLogitModel(params_all, df, null_loglik=False, pytorch=False, grad=False):

    # here we need the lenght of the parameters of each level (except the last one)
    p0 = len(LoadParameters(level0, 'Level0_UNCONSTRAINT'))
    p1 = len(LoadParameters(level1, 'Level1_UNCONSTRAINT'))
    p2 = len(LoadParameters(level2, 'Level2_UNCONSTRAINT'))

    params0 = params_all[:p0]
    params1 = params_all[p0:p0+p1]
    params2 = params_all[p0+p1:p0+p1+p2]
    params3 = params_all[p0+p1+p2:]

    df0 = prepare_data_level0(df)
    LS_m, LS_w = level0(params0, df0, logsum=True, all_sample=True)

    df['LS_m'] = LS_m
    df['LS_w'] = LS_w

    df1 = prepare_data_level1(df)
    LS_1c, LS_2c = level1(params1, df1, logsum=True, all_sample=True)
    V_BB = level1(params1, df1, utility_nestB=True, all_sample=True)

    df['LS_1c'] = LS_1c
    df['LS_2c'] = LS_2c

    df2 = prepare_data_level2(df)
    LS_car = level2(params2, df2, logsum=True, all_sample=True)

    df['LS_car'] = LS_car
    df['V_BB'] = V_BB

    df3 = prepare_data_level3(df)

    LL0 = level0(params0, df0, all_sample=True, pytorch=True, grad=grad)
    LL1 = level1(params1, df1, all_sample=True, pytorch=True, grad=grad)
    LL2 = level2(params2, df2, all_sample=True, pytorch=True, grad=grad)
    LL3 = level3(params3, df3, pytorch=True, grad=grad)

    total_LL = LL0 + LL1 + LL2 + LL3  

    if null_loglik:
        LL0 = level0(params0, df0, all_sample=True, pytorch=True, grad=grad, null_loglik=True)
        LL1 = level1(params1, df1, all_sample=True, pytorch=True, grad=grad, null_loglik=True)
        LL2 = level2(params2, df2, all_sample=True, pytorch=True, grad=grad, null_loglik=True)
        LL3 = level3(params3, df3, pytorch=True, grad=grad, null_loglik=True)

        total_LL = LL0 + LL1 + LL2 + LL3  

    return total_LL if pytorch else total_LL.detach().numpy()

### Mandatory :

- You have to start the simultaneous estimation by using the parameters that you estimated independently on each level 
- This model is highly non-linear, which means that if you start by random values, it will almost never converge and will stay stuck in a local optima 
- Also, starting with those *independetly estimated parameters* will save you a lot of time in terms of computation 

In [ ]:
# start from independent estimated parameters
all_params = torch.cat([
    LoadParameters(level0, 'Level0_UNCONSTRAINT'),
    LoadParameters(level1, 'Level1_UNCONSTRAINT'),
    LoadParameters(level2, 'Level2_UNCONSTRAINT'),
    LoadParameters(level3, 'Level3_UNCONSTRAINT')
])

In [ ]:
# If you want to start from your simultaneously estimated parameters 
# But they have to be estimated first, and then saved 
joint_parameters = JointLoadParameters(
    file_name='Joint_UNCONSTRAINT',
    level0=level0,
    level1=level1,
    level2=level2,
    level3=level3
)

In [ ]:
parameters = JointOptimize(
    func=SimultaneousNestLogitModel, 
    data=df, 
    initial_values=all_params, # start from independent estimated parameters
    gtol=1,
    max_iter=5000)

In [ ]:
# This will compute the summary table (can take some time because the model is big)
# between 1 and 2min usually
table = JointDisplayResults(
               func=SimultaneousNestLogitModel,
               params=parameters,
               data=df,
               level0=level0,
               level1=level1,
               level2=level2,
               level3=level3)

In [ ]:
JointSaveParameters(params=parameters,
                    level0=level0,
                    level1=level1,
                    level2=level2,
                    level3=level3,
                    table=table,
                    excel=True,
                    file_name='Joint_UNCONSTRAINT')